In [5]:
from openai import OpenAI
from env_utils import load_env_from_file

import json

load_env_from_file(".")

client = OpenAI()

system_prompt = """You are a legal assistant, tasked with performing the following:

First, think step by step and find the criminal legislation in the given input. You are looking for the name of the criminal offense, the criminal code/law, and the presence of European procedures EAW/EIO/REG1805 and the articles referring (possibly from other laws) to them. European procedures are always present

Second, think step by step and look in the given text see if there are names of European states present. Type only the name of the countries of "none" if there are none present

Third, think step by step and look in the given text and extract any grounds of refusal in terms of European criminal law, in a few words

Finally, think step by step and look in the given text and extract any main facts pertaining to the given case summary that might have triggered the refusal, in a few words

As an example:
Input: When validating the arrest for the execution of the European Arrest Warrant, the consent to surrender given by the person concerned is legitimate also when it follows an allegation made on the sole basis of a file in the Shengen Integrated Search System (SIS).
Output:
1) criminal legislation: none, none, EAW
2) countries: none
3) grounds of refusal: none
4) main facts: consent to surrender, Shengen Integrated Search System (SIS)"""

# system_prompt = """You are a legal assistant, tasked with segmenting a given summary of a case into different chunks. The segmentation is defined on 4 separate chunks. Each chunk is defined below and followed by an explanation of its meaning. Each definition is followed by the output format you have to follow:

# 1) criminal legislation (name of offense, criminal code, European legislation): “The case concerns the crime of [name of the offense] provided by [eg art x, criminal code] and is referred to the [EIO/EAW/REG 1805]."
# 2) countries: “The countries involved in the cooperation procedure are [x] and [y].”
# 3) relevant grounds for refusal: “In the case, the relevant ground for refusal(s) is/are [mention the ground for refusal, paying attention to use the same wording of the EU legislation].”
# 4) main facts of the case: “The main facts relevant to the case, that triggered the application (or potential application) of the ground for refusal are [x]”

# Your output must be formatted as such:
# 1) criminal legislation: [criminal offense, criminal code, European legislation]. If any are missing, replace that specific missing item with "None".
# 2) countries: [x, y].
# 3) relevant grounds for refusal: [grounds of refusal].
# 4) main facts of the case: [main facts].

# Your task is to extract information from the given summary that satisfies the definition of a chunk, or if that information is not present, to leave that chunk empty but still in the list. The format of your output should follow the same bullet-list style, but with the definitions replaced by the actual information. Keep the information you extracted as short and condensed as possible, in as few words as possible, and put it between square brackets after extraction
# """

user_messages = json.load(open("summary_test.json", "r"))["messages"]


In [8]:
responses = []


for message in user_messages:
  # r_aux = []
  # for system_prompt in system_prompt_tasks:
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo-1106",
      # model="gpt-3.5-turbo-0125",
      messages=[
        {"role": "system", "content": system_prompt},
        message
      ],
      max_tokens=512,
      frequency_penalty=0,
      top_p=1,
      temperature=0,
      presence_penalty=0
    )
    responses.append(completion)
  # responses.append(r_aux)
  
print(completion.choices[0].message.content)

1) criminal legislation: tax fraud, EIO Directive, Article 11(f), Article 4(4) of Legislative Decree No. 108 of 21 June 2017, Article 125(3) of the Criminal Procedure Code, Article 10(3) of the LD, Article 7 of the LD
2) countries: Italy, Germany
3) grounds of refusal: violation of defence rights (art. 11(f) of the EIO Directive), lack of procedural act by the executing judicial authority
4) main facts: European order for passive investigation, evidentiary seizure decree, recognition decree, motivation of the recognition decree, assessment of hindering situations, proportionality test


In [9]:
for r in responses:
    # for r in response:
        print(r.choices[0].message.content)
        print("-----")
    # print("======")

1) criminal legislation: VAT tax fraud, none, EIO, Legislative Decree No. 108 of 21 June 2017, Article 9(5)
2) countries: Italy, Germany
3) grounds of refusal: proportionality
4) main facts: VAT tax fraud, European investigation order, proportionality test, investigative needs
-----
1) criminal legislation: Law No 69 of 22 April 2005, Articles 6(1a)(d) and 18-ter, EAW
2) countries: none
3) grounds of refusal: none
4) main facts: trial held in absentia, retrial possibility, reform of conviction
-----
1) criminal legislation: tax fraud, EIO Directive, Article 11(f), Article 4(4) of Legislative Decree No. 108 of 21 June 2017, Article 125(3) of the Criminal Procedure Code, Article 10(3) of the LD, Article 7 of the LD
2) countries: Italy, Germany
3) grounds of refusal: violation of defence rights (art. 11(f) of the EIO Directive), lack of procedural act by the executing judicial authority
4) main facts: European order for passive investigation, evidentiary seizure decree, recognition decree